In [6]:
import openpyxl as opp
import pandas as pd
import numpy as np
import win32com.client
import fitz
import os
import datetime as dt

# Informacion general de las fincas y de la informacion que debe ir en cada uno de los documentos de traslado
informacion = {'transportista': ['ERICK MONTENEGRO', 'PDD3279'], #Nombre del chofer y de la placa del camion
               'fecha': dt.datetime.today().strftime('%Y-%m-%d'), #
               'aprobado_por': 'ASTRID MORENO', # Persona que autoriza el traslado
               'SM': ['AZAMA', 'JOSE LUNA'], # Nombre de la bodega y del encargado de bodega
               'MN': ['MANUELA','CAMILO GUZMAN'], # Nombre de la bodega y del encargado de bodega
               'MB': ['MARIA BONITA', 'ORLANDO FARINANGO'], # Nombre de la bodega y del encargado de bodega
               'CW':['FLORES DE LA MONTAÑA','EDWIN COBOS'], # Nombre de la bodega y del encargado de bodega
               'FS':['SANTA MONICA','JAIME CARDENAS'], # Nombre de la bodega y del encargado de bodega
               'Flopack':['Flopack',"Nelson Panchana"], # Nombre de la bodega y del encargado de bodega
               'Denmar':['DENMAR',"JAVIER CUZCO"] # Nombre de la bodega y del encargado de bodega
               }

nombre_archivo = os.path.abspath("C:\Falcon\OneDrive - GR Chia S.A.S\Excel_to_PDF\Formato_traslados.xlsx")
path = os.path.split(nombre_archivo)

# Lectura de infromacion de archivo Excel
df = pd.read_excel("Transporte Interno 2024.xlsx", sheet_name='DATOS')

# Filtra tabla por la fecha de transporte y que se pregunta en bodega
f_df = df[np.logical_and(df['FECHA']==informacion['fecha'], df['Preguntar en:']=='Bodega')]

# Obtiene las fincas que envian
informacion['up-origen'] = f_df['UP-ORIGEN'].unique()

# Obtiene las fincas que reciben
informacion['up-destino'] = f_df['UP-DESTINO'].unique()

# Obtiene las areas que estan solicitando el traslado
informacion['area_solicitante'] = f_df['Area solicitante'].unique()

<>:22: SyntaxWarning: invalid escape sequence '\F'
<>:22: SyntaxWarning: invalid escape sequence '\F'
C:\Users\cristoffer.cuyo\AppData\Local\Temp\ipykernel_17528\2776675626.py:22: SyntaxWarning: invalid escape sequence '\F'
  nombre_archivo = os.path.abspath("C:\Falcon\OneDrive - GR Chia S.A.S\Excel_to_PDF\Formato_traslados.xlsx")


In [3]:
# Muestra los nombres de las columnas que se están leyendo de la tabla de datos del archivo Excel
f_df.columns

Index(['FECHA', 'FECHA RUTA', 'FINCA ORIGEN', 'FINCA DESTINO', 'CODIGO',
       'DESCRIPCION', 'UNIDAD', 'CANTIDAD ', 'Viaje', 'Ruta', 'Preguntar en:',
       'V. Flete', 'Motivo trasaldo', 'Area solicitante', 'UP-ORIGEN',
       'UP-DESTINO', 'DIAS COBERTURA', 'Transporte', 'SEM', 'Dia', 'Mes',
       'Columna1', 'Columna2', 'Columna3'],
      dtype='object')

In [5]:
for up_origen in informacion['up-origen']:
    for up_destino in informacion['up-destino']:
        # Abre el archivo Excel, si se presenta un error y no se cierra el archivo de Excel, este programa arrojará un error de que no se puede leer el archivo
        wb = opp.load_workbook(filename = nombre_archivo)
        # Selecciona la hoja de formato de traslados para crear los PDFs en base a este formato
        sheet = wb['Traslado']
        
        # Borra el contenido de todas las celdas del formato de traslados
        for row in sheet['C11':'H23']:
            for cell in row:
                cell.value = ""

        # Se almacena el nombre del transportista
        transportista = informacion['transportista'][0]
        # Se almacena la placa del camión que va a realizar el retiro
        placa = informacion['transportista'][1]

        # En celda G4 se escribe el nombre del transportiste
        sheet['G4'] = transportista
        # En celda G6 se escribe la placa del camión
        sheet['G6'] = placa
        # En la celda G8 se escribe el nombre de la persona que aprueba el traslado, solo una persona autoriza el traslado
        sheet['G8'] = informacion['aprobado_por']

        # Se obtiene el nombre de la finca que envía
        f_envia = informacion[up_origen][0]
        # Se obtiene el nombre del nombre del bodeguero de la finca que envía
        jefe_almacen_envia = informacion[up_origen][1]

        # Se escribe en la celda G2 el nombre de la finca que envía el material
        sheet['G2'] = f_envia

        # Condicional para validar que en la iteración, la finca que envia no es la misma que la finca de destino. Ej: MN == MN
        if up_origen == up_destino:
            continue
        else:
            
            df2 = f_df[np.logical_and(f_df['UP-ORIGEN']==up_origen, f_df['UP-DESTINO']==up_destino)] # Filtra los valores para las fincas de origen y destino

            if len(df2) == 0: # Si el dataframe filtrado no tiene elementos, pasa a la siguiente iteración
                continue
            
            elif len(df2) > 13: # Condicional para validar que el número de artículos trasladados no sean mayor a 13. Si hay más de 13 artículos que se envían el formato no está diseñado para esto
                print("Error: revisar la cantidad de articulos que van a ser traslados y notificar si hay cambio de formato")
                break
            
            else:
                f_solicita = informacion[up_destino][0]
                jefe_almacen_solicita = informacion[up_destino][1]
                # En celda D2 se escribe la finca que solicita
                sheet['D2'] = f_solicita
                # En celda D4 se escribe el nombre del encargado de Bodega que debe recibir el material
                sheet['D4'] = jefe_almacen_solicita
                # En celda D6 se escribe el area solicitante
                sheet['D6'] = informacion['area_solicitante'][0]
                # En celda C35 se escribe el nombre del encargado de bodega que envía los materiales
                sheet['C35'] = f"NOMBRE: {informacion[up_origen][1]}"
                # Numero de la celda donde se puede empezar a escribir en el formato de traslados
                celda_incio = 11
                # Variable para que se mueva entre valores del dataframe filtrado
                indice = 0

                for i in range(len(df2)): # Escribe en cada una de las columnas los valores de cada traslado
                    sheet[f'C{celda_incio}'] = df2.iloc[indice,1]
                    sheet[f'D{celda_incio}'] = df2.iloc[indice,4]
                    sheet[f'E{celda_incio}'] = df2.iloc[indice,5]
                    sheet[f'F{celda_incio}'] = df2.iloc[indice,7]
                    sheet[f'G{celda_incio}'] = df2.iloc[indice,6]
                    sheet[f'H{celda_incio}'] = df2.iloc[indice,12]
                    celda_incio += 1
                    indice += 1
                #Se guarda el archivo de formato de traslados con el mismo nombre
                wb.save(filename= nombre_archivo)
                # Se cierra el archivo. Si el programa se detiene antes de llegar a esta línea, se deberá reiniciar la PC debido al error que arroja
                wb.close()

                # Open Microsoft Excel 
                excel = win32com.client.Dispatch("Excel.Application") 
                excel.Visible = True

                # Read Excel File 
                sheets = excel.Workbooks.Open(nombre_archivo)
                # Nombre del archivo PDF
                name_pdf = f"TRASLADO_{up_origen}-{up_destino}.pdf"
                # Guardado del Excel en foramato PDF
                sheets.ExportAsFixedFormat(0, os.path.join(path[0],name_pdf))
                # Se cierra el archivo Excel
                sheets.Close()
                # Se sale de la aplicación Excel
                excel.Quit()
               
                # Open the PDF document, using raw strings for file paths
                doc = fitz.open(os.path.join(path[0],name_pdf))

                # Access the first page
                page = doc[0]

                # Create a rectangle for image placement, ensuring correct dimensions
                rect = fitz.Rect(40, 320, 300, 400)  # Si se cambia el formato, llamar a alguien a que corrija la posición de la firma

                # Insert the image into the rectangle
                page.insert_image(
                    rect,
                    filename="SUMILLA-removebg-preview.png" # Nombre de la foto de la firma que se va a insertar en el PDF
                )

                # Save the modified PDF with a new filename
                name2 = f"TRASLADO {up_origen}-{up_destino}-signed.pdf"
                doc.save(filename=os.path.join(path[0],name2))
